## Init

In [1]:
import glob
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as f
import torch.nn.functional as func
from PIL import Image
from skimage.metrics import structural_similarity
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from pytorchtools import EarlyStopping
import scipy
import scipy.io
import matplotlib
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
from tqdm import tqdm

init_notebook_mode(connected=True)
%matplotlib inline

In [2]:
result = []

epochs = 10000
interval = 100
dropout_p = .0
NUM_CLASSES = 8
NUM_ROUTING_ITERATIONS = 3
device = torch.device('cuda')
# random_seed = 1
# torch.manual_seed(random_seed)
# torch.cuda.manual_seed(random_seed)
# np.random.seed(random_seed)

batch_size = 32
lr = 1e-3
wd = 1e-7

In [3]:
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)

    def forward(self, x):
        x = self.fc1(x)
        x = f.relu(x)
        # x = f.dropout(x, p=dropout_p, training=self.training)
        # x = self.bn1(x)
        x = self.fc2(x)
        x = f.relu(x)
        # x = f.dropout(x, p=dropout_p, training=self.training)
        # x = self.bn2(x)
        x = self.fc3(x)
        x = f.relu(x)
        # x = f.dropout(x, p=dropout_p, training=self.training)
        # x = self.bn3(x)
        x = self.fc4(x)
        # x = x.view(-1, 8, 16)
        # x = self.squash(x)
        # x = x.view(-1, 8 * 16)
        return x

    def squash(self, tensor, dim=-1):
        squared_norm = (tensor ** 2).sum(dim=dim, keepdim=True)
        scale = squared_norm / (1 + squared_norm)
        return scale * tensor / torch.sqrt(squared_norm)

In [4]:
def squash(tensor, dim=-1):
    squared_norm = (tensor ** 2).sum(dim=dim, keepdim=True)
    scale = squared_norm / (1 + squared_norm)
    return scale * tensor / torch.sqrt(squared_norm)

In [5]:
def train(epoch):
    # Set model to training mode
    model.train()
    # Loop over each batch from the training set
    for batch_idx, (data, target) in enumerate(train_loader):
        # Copy data to GPU if needed
        data = data.to(device)
        target = target.to(device)
        # Zero gradient buffers
        optimizer.zero_grad()
        # Pass data through the network
        output = model(data)
        # Calculate loss
        loss = criterion(output, target)

        l2_reg = None
        for name, param in model.named_parameters():
            if not (name == 'fc3.bias' or name == 'fc3.weight'):
                if l2_reg is None:
                    l2_reg = 0.5 * param.norm(2)**2
                else:
                    l2_reg += 0.5 * param.norm(2)**2
        loss += l2_reg * wd
        # Backpropagate
        loss.backward()
        # Update weights
        optimizer.step()

    if epoch % (interval/100) == 1:
#         writer.add_scalar('loss/train', loss.data.item(), epoch)
        print('Epoch: {} Train Loss: {:.6f}'.format(epoch, loss.data.item()))

In [6]:
def validation(epoch):
    model.eval()
    val_loss = 0
    for data, target in validation_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        val_loss += criterion(output, target).data.item()

    val_loss /= len(validation_loader)

    if epoch % (interval/100) == 1:
#         writer.add_scalar('loss/valid', val_loss, epoch)
        print('Epoch: {} Valid Loss: {:.6f}'.format(epoch, val_loss))
    return output, val_loss

In [7]:
def validation_on_train(epoch):
    model.eval()
    val_loss = 0
    for data, target in train_for_validation_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
    return output

In [8]:
def softmax(input, dim=1):
    transposed_input = input.transpose(dim, len(input.size()) - 1)
    softmaxed_output = func.softmax(transposed_input.contiguous().view(-1, transposed_input.size(-1)), dim=-1)
    return softmaxed_output.view(*transposed_input.size()).transpose(dim, len(input.size()) - 1)

In [9]:
class CapsuleLayer(nn.Module):
    def __init__(self, num_capsules, num_route_nodes, in_channels, out_channels, kernel_size=None, stride=None,
                 num_iterations=NUM_ROUTING_ITERATIONS):
        super(CapsuleLayer, self).__init__()

        self.num_route_nodes = num_route_nodes
        self.num_iterations = num_iterations

        self.num_capsules = num_capsules

        if num_route_nodes != -1:
            self.route_weights = nn.Parameter(torch.randn(num_capsules, num_route_nodes, in_channels, out_channels))
        else:
            self.capsules = nn.ModuleList(
                [nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=0) for _ in
                 range(num_capsules)])

    def squash(self, tensor, dim=-1):
        squared_norm = (tensor ** 2).sum(dim=dim, keepdim=True)
        scale = squared_norm / (1 + squared_norm)
        return scale * tensor / torch.sqrt(squared_norm)

    def forward(self, x):
        if self.num_route_nodes != -1:
            priors = x[None, :, :, None, :] @ self.route_weights[:, None, :, :, :]

            logits = Variable(torch.zeros(*priors.size())).to(device)
            for i in range(self.num_iterations):
                probs = softmax(logits, dim=2)
                outputs = self.squash((probs * priors).sum(dim=2, keepdim=True))

                if i != self.num_iterations - 1:
                    delta_logits = (priors * outputs).sum(dim=-1, keepdim=True)
                    logits = logits + delta_logits
        else:
            outputs = [capsule(x).view(x.size(0), -1, 1) for capsule in self.capsules]
            outputs = torch.cat(outputs, dim=-1)
            outputs = self.squash(outputs)

        return outputs

In [10]:
class CapsuleNet(nn.Module):
    def __init__(self):
        super(CapsuleNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=256, kernel_size=9, stride=1)
        self.primary_capsules = CapsuleLayer(num_capsules=8, num_route_nodes=-1, in_channels=256, out_channels=32,
                                             kernel_size=9, stride=2)
        self.digit_capsules = CapsuleLayer(num_capsules=NUM_CLASSES, num_route_nodes=32 * 6 * 6, in_channels=8,
                                           out_channels=16)
        self.decoder = nn.Sequential(
            nn.Linear(16 * NUM_CLASSES, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 784),
            nn.Sigmoid()
        )

    def forward(self, x, y=None):
        # x = func.relu(self.conv1(x), inplace=True)
        # x = self.primary_capsules(x)
        # x = self.digit_capsules(x).squeeze().transpose(0, 1)
        classes = (x ** 2).sum(dim=-1) ** 0.5
        classes = func.softmax(classes, dim=-1)
        if y is None:
            # In all batches, get the most active capsule.
            _, max_length_indices = classes.max(dim=1)
            y = Variable(torch.eye(NUM_CLASSES)).to(device).index_select(dim=0, index=max_length_indices.data)
        reconstructions = self.decoder((x * y[:, :, None]).view(x.size(0), -1))
        return classes, reconstructions

In [11]:
def get_iterator():
    data0 = []
    labels = []
    char_labels = []
    files = glob.glob('stim/*')
    files = np.sort(files)
    for i in range(0, len(files)):
        im = Image.open(files[i])
        im.load()
        data0.append(np.asarray(im))
        labels.append(i)
        name = files[i]
        char_labels.append(int(name[5]))
    data0 = np.asarray(data0)
    data0 = data0[:, 2:30, 2:30]
    labels = np.asarray(labels)
    data0 = np.expand_dims(data0, axis=1)
    data = data0
    for _ in range(7):
        data = np.concatenate([data, data0], axis=0)
    return data, char_labels

## Train

In [ ]:
for iSave in range(31,100):
    cntAll = 5*7*2
    cnt = 0
    for roi in ['v1', 'v2', 'v3', 'v1.v2', 'v1.v2.v3']:
        for subs in [['sub01'], ['sub02'], ['sub03'], ['sub04'], ['sub06'], ['sub07'], ['sub08']]:
            for sess in [['ses01'], ['ses02']]:
                cnt = cnt + 1
                print(int(cnt/cntAll*100))
                x = []
                for iSub in range(len(subs)):
                    for iSes in range(len(sess)):
                        sub = subs[iSub]
                        ses = sess[iSes]
                        x.append(np.genfromtxt('./probe/probe.' + sub + '.' + ses + '.' + roi + '.txt', delimiter=','))
                x = np.asarray(x)
                x = x.reshape((-1, 128))

                y0 = np.load('digitcaps.npy')
                y0 = y0.reshape((128, 8*16))
                y = y0
                for i in range(7):
                    y = np.concatenate([y, y0], axis=0)
                input_size = x.shape[1]

                rnd = np.random.permutation(len(x))
                x = x[rnd, :]
                y = y[rnd, :]

                x = x - 100
                train_num = int(len(x) * 0.75)
                valid_num = len(x) - train_num
                x_train = torch.from_numpy(x[:train_num, :])
                y_train = torch.from_numpy(y[:train_num, :])
                x_valid = torch.from_numpy(x[train_num:, :])
                y_valid = torch.from_numpy(y[train_num:, :])

                train_dataset = TensorDataset(x_train.float(), y_train.float())
                validation_dataset = TensorDataset(x_valid.float(), y_valid.float())

                train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
                train_for_validation_loader = DataLoader(dataset=train_dataset, batch_size=train_num, shuffle=False)
                validation_loader = DataLoader(dataset=validation_dataset, batch_size=valid_num, shuffle=False)

                model = Net(input_size).to(device)
                optimizer = torch.optim.Adam(model.parameters(), lr=lr)
                criterion = nn.MSELoss()

                capsnet = CapsuleNet().to(device)
                capsnet.load_state_dict(torch.load('stage1/epochs/epoch_500.pt'))

                rndTrainArg = np.argsort(rnd[:train_num])
                rndValidArg = np.argsort(rnd[-valid_num:])
                rndTrain = rnd[:train_num]
                rndValid = rnd[-valid_num:]
                data, char_labels = get_iterator()
                char_labels = np.asarray(char_labels)
                ground_truth_train = torch.tensor((data[rnd[:train_num], :, :]) / 255.0)
                ground_truth_valid = torch.tensor((data[rnd[-valid_num:], :, :]) / 255.0)
                char_labels_train = char_labels[rnd[:train_num]]
                char_labels_valid = char_labels[rnd[-valid_num:]]

                early_stopping = EarlyStopping(patience=500, verbose=False, filename='checkpoint.pt')

                for epoch in range(1, epochs + 1):
                    train(epoch)

                    # early_stopping(val_loss, model)
                    # if early_stopping.early_stop:
                    #     print("Early stopping")
                    #     break

                    if epoch == epochs:
                        outValid, val_loss = validation(epoch)
                        outTrain = validation_on_train(epoch)

                        outValid = outValid.reshape((valid_num, 8, 16))
                        reconstructions = capsnet.forward(torch.tensor(squash(outValid)).to(device))[1]
                        reconstruction_valid = reconstructions.cpu().view_as(ground_truth_valid).data

                        # SSIM
                        ground_truth_valid_numpy = ground_truth_valid.float().cpu().detach().numpy()
                        reconstruction_valid_numpy = reconstruction_valid.float().cpu().detach().numpy()
                        ssim = []
                        for i in range(len(ground_truth_valid)):
                            ssim.append(structural_similarity(ground_truth_valid_numpy[i, 0, :, :], reconstruction_valid_numpy[i, 0, :, :]))

                        for i in range(len(ground_truth_valid_numpy)):
                            A = ground_truth_valid_numpy[i, 0, :, :]
                            B = reconstruction_valid_numpy[i, 0, :, :]
                            result.append({'ssim': ssim[i],
                                           'mse': ((A - B)**2).mean(axis=None),
                                           'pcc': np.corrcoef(A.flat, B.flat)[0, 1],
                                           'index': rndValid[i],
                                           'char_index': char_labels_valid[i],
                                           'kind_index': int(rndValid[i] % 16) + 1,
                                           'type': 'valid',
                                           'roi': roi,
                                           'sub': subs,
                                           'sess': sess,
                                           'original': A,
                                           'reconstruction': B,
                                           })

                        outTrain = outTrain.reshape((train_num, 8, 16))
                        reconstructions = capsnet.forward(torch.tensor(squash(outTrain)).to(device))[1]
                        reconstruction_train = reconstructions.cpu().view_as(ground_truth_train).data

                        # SSIM
                        ground_truth_train_numpy = ground_truth_train.float().cpu().detach().numpy()
                        reconstruction_train_numpy = reconstruction_train.float().cpu().detach().numpy()
                        ssim = []
                        for i in range(len(ground_truth_train)):
                            ssim.append(structural_similarity(ground_truth_train_numpy[i, 0, :, :], reconstruction_train_numpy[i, 0, :, :]))

                        for i in range(len(ground_truth_train_numpy)):
                            A = ground_truth_train_numpy[i, 0, :, :]
                            B = reconstruction_train_numpy[i, 0, :, :]
                            result.append({'ssim': ssim[i],
                                           'mse': ((A - B) ** 2).mean(axis=None),
                                           'pcc': np.corrcoef(A.flat, B.flat)[0, 1],
                                           'index': rndTrain[i],
                                           'char_index': char_labels_train[i],
                                           'kind_index': int(rndTrain[i] % 16) + 1,
                                           'type': 'train',
                                           'roi': roi,
                                           'sub': subs,
                                           'sess': sess,
                                           'original': A,
                                           'reconstruction': B,
                                           })
    np.savez_compressed('result'+str(iSave)+'.npz', data=result)

## Plot

In [402]:
AllData = [None]*48
for i in tqdm(range(1,48)):
    AllData[i] = np.load('result'+str(i)+'.npz' ,allow_pickle=True)['data']
AllData[0] = np.load('result.npz' ,allow_pickle=True)['data']

100%|██████████| 47/47 [02:50<00:00,  3.63s/it]


In [417]:
result = pd.DataFrame(data=[], columns=['ssim', 'mse', 'pcc', 'index', 'char_index', 'kind_index', 'type', 'roi', 'sub', 'sess', 
                                                  'original', 'reconstruction'])
for i in tqdm(range(len(AllData))):
    temp = pd.DataFrame(data=list(AllData[i]), columns=result.columns)
    result = pd.concat((result,temp))
result['sub'] = result['sub'].apply(lambda x: x[0])
result['sess'] = result['sess'].apply(lambda x: x[0])

100%|██████████| 48/48 [00:17<00:00,  2.81it/s]


In [87]:
# 9 12 13 14* 16
result = np.load('result9.npz' ,allow_pickle=True)['data']
result = pd.DataFrame(data=list(result), columns=['ssim', 'mse', 'pcc', 'index', 'char_index', 'kind_index', 'type', 'roi', 'sub', 'sess', 
                                                  'original', 'reconstruction'])
result['sub'] = result['sub'].apply(lambda x: x[0])
result['sess'] = result['sess'].apply(lambda x: x[0])
result

,ssim,mse,pcc,index,char_index,kind_index,type,roi,sub,sess,original,reconstruction
0,0.280082,0.116794,0.396538,77,5,14,valid,v1,sub01,ses01,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.14514622, 0.13635553, 0.15237167, 0.111839..."
1,0.085886,0.154417,0.316807,117,8,6,valid,v1,sub01,ses01,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.045423757, 0.04474354, 0.04651954, 0.05333..."
2,-0.137922,0.247081,-0.196325,42,3,11,valid,v1,sub01,ses01,"[[0.0, 0.0, 0.023529412, 0.05490196, 0.0313725...","[[0.13089384, 0.14468645, 0.16840878, 0.184025..."
3,0.202052,0.141553,0.395746,104,7,9,valid,v1,sub01,ses01,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0039215...","[[0.04094705, 0.03890289, 0.037331842, 0.04291..."
4,0.151082,0.158080,0.313537,31,2,16,valid,v1,sub01,ses01,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.09491834, 0.10840305, 0.120091, 0.12836982..."
...,...,...,...,...,...,...,...,...,...,...,...,...
62715,0.636946,0.050256,0.811386,123,8,12,train,v1.v2.v3,sub08,ses02,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.00050751294, 0.00042475297, 0.000787431, 0..."
62716,0.530987,0.063632,0.672481,52,4,5,train,v1.v2.v3,sub08,ses02,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.010810369, 0.012419895, 0.015142259, 0.014..."
62717,0.583206,0.068066,0.742746,46,3,15,train,v1.v2.v3,sub08,ses02,"[[0.043137256, 0.015686275, 0.0, 0.0, 0.0, 0.0...","[[0.047192134, 0.050334424, 0.05594229, 0.0422..."
62718,0.194461,0.117968,0.489174,85,6,6,train,v1.v2.v3,sub08,ses02,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.019817842, 0.015434144, 0.022351732, 0.036..."


In [88]:
result['value'] = np.nan
result.loc[(result['sub'].isin(['sub01','sub02','sub03','sub04'])) & result['char_index'].isin([1,2,3,4]), ['value']] = 1
result.loc[(result['sub'].isin(['sub01','sub02','sub03','sub04'])) & result['char_index'].isin([5,6,7,8]), ['value']] = 0
result.loc[(result['sub'].isin(['sub05','sub06','sub07','sub08'])) & result['char_index'].isin([1,2,3,4]), ['value']] = 0
result.loc[(result['sub'].isin(['sub05','sub06','sub07','sub08'])) & result['char_index'].isin([5,6,7,8]), ['value']] = 1

In [114]:
prop = 'ssim'
PreGood  = result.loc[(result['type'] == 'valid') & (result['value'] == 1) & (result['sess'] == 'ses01') & (result['roi'] == 'v1')]
PreBad   = result.loc[(result['type'] == 'valid') & (result['value'] == 0) & (result['sess'] == 'ses01') & (result['roi'] == 'v1')]
PostGood = result.loc[(result['type'] == 'valid') & (result['value'] == 1) & (result['sess'] == 'ses02') & (result['roi'] == 'v1')]
PostBad  = result.loc[(result['type'] == 'valid') & (result['value'] == 0) & (result['sess'] == 'ses02') & (result['roi'] == 'v1')]

PreGoodBest  = result.loc[(result['type'] == 'valid') & (result['value'] == 1) & (result['sess'] == 'ses01') & (result['roi'] == 'v1')]
PreGoodBest = PreGoodBest.reindex(PreGoodBest['ssim'].sort_values().index[::-1])
PreBadBest   = result.loc[(result['type'] == 'valid') & (result['value'] == 0) & (result['sess'] == 'ses01') & (result['roi'] == 'v1')]
PreBadBest = PreBadBest.reindex(PreBadBest['ssim'].sort_values().index[::-1])
PostGoodBest = result.loc[(result['type'] == 'valid') & (result['value'] == 1) & (result['sess'] == 'ses02') & (result['roi'] == 'v1')]
PostGoodBest = PostGoodBest.reindex(PostGoodBest['ssim'].sort_values().index[::-1])
PostBadBest  = result.loc[(result['type'] == 'valid') & (result['value'] == 0) & (result['sess'] == 'ses02') & (result['roi'] == 'v1')]
PostBadBest = PostBadBest.reindex(PostBadBest['ssim'].sort_values().index[::-1])


In [115]:
# ind = result.loc[(result['type'] == 'valid') & (result['value'] == 1)]['ssim'].sort_values().index[::-1]
# best = result.loc[(result['type'] == 'valid') & (result['value'] == 1)].reindex(ind) 
# im = []
# for i in range(16,31):
#     im.append(np.concatenate((best['original'].iloc[i],best['reconstruction'].iloc[i])))

# fig = px.imshow(np.concatenate(np.array(im), axis=1), color_continuous_scale='gray')
# fig.update_layout(coloraxis_showscale=False)
# fig.update_xaxes(showticklabels=False)
# fig.update_yaxes(showticklabels=False)
# fig.show()

In [116]:
from scipy import stats
Pre = stats.ttest_ind(PreGood[prop], PreBad[prop])
Post = stats.ttest_ind(PostGood[prop], PostBad[prop])
Good = stats.ttest_ind(PreGood[prop], PostGood[prop])
Bad = stats.ttest_ind(PreBad[prop], PostBad[prop])
matrix_twosample = [
    ['', 'Test Statistic', 'p-value'],
    ['Pre', Pre[0], Pre[1]],
    ['Post', Post[0], Post[1]],
    ['Good', Good[0], Good[1]],
    ['Bad', Bad[0], Bad[1]],
]

twosample_table = plotly.tools.FigureFactory.create_table(matrix_twosample, index=True)
twosample_table

In [117]:
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Good',
    x=['Pre', 'Post'], y=[PreGood[prop].mean(), PostGood[prop].mean()],
    error_y=dict(type='data', array=[PreGood[prop].std()/np.sqrt(PreGood[prop].count()), PreBad[prop].std()/np.sqrt(PreBad[prop].count())])
))
fig.add_trace(go.Bar(
    name='Bad',
    x=['Pre', 'Post'], y=[PreBad[prop].mean(), PostBad[prop].mean()],
    error_y=dict(type='data', array=[PostGood[prop].std()/np.sqrt(PostGood[prop].count()), PostBad[prop].std()/np.sqrt(PostBad[prop].count())])
))
fig.update_layout(title={'text':'SSIM V1','y':0.9,'x':0.5,'xanchor':'center','yanchor':'top'},
                  yaxis=dict(range=[0, 0.12]), barmode='group',width=400)
fig.show()

In [120]:
Till = 70
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Good',
    x=['Pre', 'Post'], y=[PreGoodBest.iloc[:Till][prop].mean(), PostGoodBest.iloc[:Till][prop].mean()],
    error_y=dict(type='data', array=[PreGoodBest.iloc[:Till][prop].std()/np.sqrt(PreGoodBest.iloc[:Till][prop].count()), PreBadBest.iloc[:Till][prop].std()/np.sqrt(PreBadBest.iloc[:Till][prop].count())])
))
fig.add_trace(go.Bar(
    name='Bad',
    x=['Pre', 'Post'], y=[PreBadBest.iloc[:Till][prop].mean(), PostBadBest.iloc[:Till][prop].mean()],
    error_y=dict(type='data', array=[PostGoodBest.iloc[:Till][prop].std()/np.sqrt(PostGoodBest.iloc[:Till][prop].count()), PostBad[prop].std()/np.sqrt(PostBad[prop].count())])
))
fig.update_layout(title={'text':'SSIM V1 Best','y':0.9,'x':0.5,'xanchor':'center','yanchor':'top'},
                  yaxis=dict(range=[0.25, 0.35]), barmode='group',width=400)
fig.show()

In [119]:
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Pre',
    x=['Good', 'Bad'], y=[PreGood[prop].mean(), PreBad[prop].mean()],
    error_y=dict(type='data', array=[PreGood[prop].std()/np.sqrt(PreGood[prop].count()), PreBad[prop].std()/np.sqrt(PreBad[prop].count())])
))
fig.add_trace(go.Bar(
    name='Post',
    x=['Good', 'Bad'], y=[PostGood[prop].mean(), PostBad[prop].mean()],
    error_y=dict(type='data', array=[PostGood[prop].std()/np.sqrt(PostGood[prop].count()), PostBad[prop].std()/np.sqrt(PostBad[prop].count())])
))
fig.update_layout(title={'text':'SSIM V1','y':0.9,'x':0.5,'xanchor':'center','yanchor':'top'},
                  yaxis=dict(range=[0, 0.12]), barmode='group',width=400)
fig.show()

In [ ]:
trace0 = go.Histogram(x=PreGood[prop], name='PreGood', opacity=0.75)
trace1 = go.Histogram(x=PreBad[prop], name='PreBad', opacity=0.75)
layout = go.Layout(barmode='overlay', title='Pre')
fig = go.Figure(data=[trace0, trace1], layout=layout)
fig.show()
trace0 = go.Histogram(x=PostGood[prop], name='PostGood', opacity=0.75)
trace1 = go.Histogram(x=PostBad[prop], name='PostBad', opacity=0.75)
layout = go.Layout(barmode='overlay', title='Post')
fig = go.Figure(data=[trace0, trace1], layout=layout)
fig.show()
trace0 = go.Histogram(x=PreGood[prop], name='PreGood', opacity=0.75)
trace1 = go.Histogram(x=PostGood[prop], name='PostGood', opacity=0.75)
layout = go.Layout(barmode='overlay', title='Good')
fig = go.Figure(data=[trace0, trace1], layout=layout)
fig.show()
trace0 = go.Histogram(x=PostGood[prop], name='PreBad', opacity=0.75)
trace1 = go.Histogram(x=PostBad[prop], name='PostBad', opacity=0.75)
layout = go.Layout(barmode='overlay', title='Bad')
fig = go.Figure(data=[trace0, trace1], layout=layout)
fig.show()

In [602]:
prop = 'ssim'
Mean = result.loc[result['type'] == 'valid'].groupby('roi').mean()[prop]
SEM = result.loc[result['type'] == 'valid'].groupby('roi').std()[prop]/result.loc[result['type'] == 'valid'].groupby('roi').count()[prop].apply(np.sqrt)
fig = go.Figure()
fig.add_trace(go.Bar(x=Mean.index, y=Mean, error_y=dict(type='data', array=SEM)))
fig.update_layout(title={'text':'ROI','y':0.9,'x':0.5,'xanchor':'center','yanchor':'top'}, width=600)
fig.show()

In [597]:
prop = 'ssim'
Mean = result.loc[result['type'] == 'valid'].groupby('sub').mean()[prop].sort_values()
SEM = result.loc[result['type'] == 'valid'].groupby('sub').std()[prop].reindex(Mean.index)/result.loc[result['type'] == 'valid'].groupby('sub').count()[prop].apply(np.sqrt).reindex(Mean.index)
fig = go.Figure()
fig.add_trace(go.Bar(x=Mean.index, y=Mean, error_y=dict(type='data', array=SEM)))
fig.update_layout(title={'text':'Subject','y':0.9,'x':0.5,'xanchor':'center','yanchor':'top'}, width=600)
fig.show()

In [590]:
prop = 'ssim'
Mean = result.loc[result['type'] == 'valid'].groupby('char_index').mean()[prop].sort_values()
SEM = result.loc[result['type'] == 'valid'].groupby('char_index').std()[prop].reindex(Mean.index)/result.loc[result['type'] == 'valid'].groupby('char_index').count()[prop].apply(np.sqrt).reindex(Mean.index)
fig = go.Figure()
fig.add_trace(go.Bar(y=Mean, error_y=dict(type='data', array=SEM)))
fig.update_layout(title={'text':'Char','y':0.9,'x':0.5,'xanchor':'center','yanchor':'top'}, width=600)
fig.show()

In [596]:
prop = 'ssim'
Mean = result.loc[result['type'] == 'valid'].groupby('sess').mean()[prop]
SEM = result.loc[result['type'] == 'valid'].groupby('sess').std()[prop]/result.loc[result['type'] == 'valid'].groupby('sess').count()[prop].apply(np.sqrt)
fig = go.Figure()
fig.add_trace(go.Bar(x=Mean.index, y=Mean, error_y=dict(type='data', array=SEM)))
fig.update_layout(title={'text':'Session','y':0.9,'x':0.5,'xanchor':'center','yanchor':'top'}, width=300)
fig.show()

In [599]:
prop = 'ssim'
Mean = result.groupby('type').mean()[prop]
SEM = result.groupby('type').std()[prop]/result.groupby('type').count()[prop].apply(np.sqrt)
fig = go.Figure()
fig.add_trace(go.Bar(x=Mean.index, y=Mean, error_y=dict(type='data', array=SEM)))
fig.update_layout(title={'text':'Type','y':0.9,'x':0.5,'xanchor':'center','yanchor':'top'}, width=300)
fig.show()